# Time Intelligence in Power BI


> Time intelligence in Power BI refers to a set of functions that perform calculations based on time periods. These functions enable analyses like year-to-date totals, moving averages, and year-over-year comparisons, providing essential insights for datasets with a temporal dimension.

## Follow Along with the Examples

Time Intelligence functions are easier to learn if you put them into practice. To follow along with the examples in this lesson: 

1. Download this [`csv` file](https://cdn.theaicore.com/content/projects/e71908f1-fbf5-4c65-ad36-7d43589c3706/timeintelligence_sales_data.csv)
2. Load it into a new Power Bi Desktop project (**Home** > **Get Data** > **Text/CSV**)
3. Rename the table to `Sales`
4. Create a new calculated column called `Sales amount`, which is `Sales[Order quantity]` * `Sales[Unit price]`



## Configuring Date Tables

Date tables form the basis for most time intelligence functions in Power BI. Proper configuration ensures that your time-based calculations are both accurate and consistent. 

### Importance of a Continuous Date Table

>In many datasets, dates might appear sporadically, perhaps representing only transaction dates or specific event days. While this might seem sufficient, it can lead to misleading insights, especially when looking at periods without data. A continuous date table fills the gaps, ensuring every date within a range has a representation, even if there's no direct data for some dates.

For instance, if your sales data skips weekends because no transactions were made, you might get an incomplete view of a month's total days. But with a continuous date table, weekends are accounted for, and your analyses, such as average daily sales, remain accurate.


### Marking a Date Table in Power BI

Once you have a complete date table, it's important to designate it correctly within Power BI.

1. Go to the **Data view** in Power BI Desktop
2. Select the date table
3. From the **Table Tools** tab, choose **Mark as Date Table**
4. Specify the unique date column. This will either be the date itself, or a date UUID column

By marking a table as a date table, you inform Power BI of your intention to use it for time intelligence. This step ensures optimal performance during calculations and guarantees compatibility with all time intelligence functions. Always remember to define a unique date column without any missing, duplicate, or invalid dates.


## Creating a Date Table from Scratch

Having a comprehensive date table is paramount for effective time intelligence in Power BI. While there are many ways to import date tables, creating one directly in Power BI provides flexibility and ensures compatibility. Let's walk through some methods to construct your own.

### Using `CALENDAR` Function

The `CALENDAR` function is a straightforward way to create a date table. It requires two parameters: a start date and an end date.

1. Navigate to the **New Table** icon on the **Home** pane of the ribbon
2. Enter the following DAX formula:

```dax
Dates = CALENDAR(DATE(2020,1,1), DATE(2023,12,31))
```

This will generate a table with dates from January 1, 2020, to December 31, 2022. Every single day within this range is accounted for.

<p align="center">
<img src="images/create_datetable.gif" width="900"> <br> <br>
</p>

Once you have created the table in your project, you can delete it as we will use the one below instead.

### Creating a Custom Date Range with `DATESBETWEEN`

Sometimes, you might need a custom range, not strictly defined by a clear start and end. The `DATESBETWEEN` function provides this flexibility.

Assuming you have a sales table and you want a date range from the first sale to a specific end date:

```dax
Dates = DATESBETWEEN(
    Sales[Order Date], 
    MIN(Sales[Order Date]), 
    DATE(2023,12,31)
)
```


Create this table in your project, and create a connection between it and `Sales[Order Date]`. We will use it elsewhere in the lesson.

<p align="center">
<img src="images/connect_datetable.gif" width="900"> <br> <br>
</p>


## Building Out a Date Table

As you begin to design a report, it will often be necessary to add more columns to your date table to facilitate the different types of analysis you wish to perform. For example, if you wish to analyse revenue per quarter, you will want a column that gives the start date of the quarter to which each date belongs. Power BI has specific functions to help generate most of these expansions to the date table. 

### `STARTOFYEAR`, `STARTOFQUARTER`, `STARTOFMONTH`

These functions all work the same way, taking in a range of dates and returning the first day of the relevant period. For example, to create a `[StartOfYear]` column we can do:

```dax
StartOfYear = STARTOFYEAR(Dates[Date])
```

<p align="center">
<img src="images/startofyear.gif" width="900"> <br> <br>
</p>

You can use the same measure to create the quarter and month columns.

#### Creating a Start of Week Column

Unfortunately for some reason Power BI does not have a `STARTOFWEEK` function. You can however generate this leveraging a function called `WEEKDAY`, which returns the day of the week as an integer.

```dax
StartOfWeek = Dates[Date] - WEEKDAY(Dates[Date],2) + 1
```

This is subtracting the day of the week from the specified date, and adding one. By default `WEEKDAY` uses Sunday as the start of the week. The `2` in the second argument of `WEEKDAY` tells it to use Monday instead.

<p align="center">
<img src="images/startofweek.gif" width="900"> <br> <br>
</p>



## Generating Other Fields in the Date Table

The precise set of fields you will need to add to your date table will depend on your analysis goals. Here are a selection that will be used regularly. For a full list you can check the Power BI [documentation](https://learn.microsoft.com/en-us/dax/time-intelligence-functions-dax).

### `QUARTER`

```dax
Quarter = QUARTER(Dates[Date])
```

Returns the quarter as an integer.

### `WEEKNUM`

Returns the week of the year for the input date range.

```dax
WeekNumber = WEEKNUM(Dates[Date])
```

### `ENDOFMONTH`, `ENDOFQUARTER`, `ENDOFYEAR`

Work the same as `STARTOFYEAR`, but return the last day of the period instead.

## Basic Time Intelligence Functions

Time intelligence functions in Power BI offer a suite of options for analyzing data over different time periods. This section provides an introduction to some fundamental functions you'll frequently encounter.


### `TOTALYTD`, `TOTALQTD`, and `TOTALMTD`

These functions compute an expression from the start of the current time period to the current point in the time period (year, quarter and month respectively). They all have the same general formula, for example:

```dax
TOTALYTD(<expression>,<dates>,[<filter>])  
```
Each function requires at least the measure to aggregate and the date column from your date table. The filter argument is optional.

Here is an example usage, to create a measure for the total sales amount for the current year:
   
```dax
SalesYTD = TOTALYTD(SUM(Sales[Sales amount]), Sales[Order Date])
```

Here is an example visual using the `[SalesYTD]` measure we have just created:

<p align="center">
<img src="images/sales_ytd_graph.png" width="900"> <br> <br>
</p>



## Time-based Filtering

In Power BI, not only can you calculate aggregates over specific time frames, but you can also filter your data based on time periods. Here are some of the more widely-used time-based filtering functions:




### `DATEADD` and `EDATE`

Both `DATEADD` and `EDATE` functions allow you to filter to dates shifted by a specific period, enabling analyses like year-over-year or month-over-month comparisons.

- **`DATEADD`**: This function shifts a set of dates by a specified number of intervals

```dax
PreviousYearSales = CALCULATE(
    SUM(Sales[Sales amount]),
    DATEADD(Dates[Date], -1, YEAR)
)
```
This can be combined with a `[TotalSales]` measure to give a year-on-year percentage change:

```dax
TotalSales = SUM(Sales[Sale amount])
```

```dax
YearOnYearPercentChange = DIVIDE(TotalSales,PreviousYearSales) * 100
```

<p align="center">
<img src="images/yoychange.png" width="900"> <br> <br>
</p>

- **`EDATE`**: While `DATEADD` works for various intervals (like year, quarter, and day), `EDATE` specifically shifts months. It's especially useful when working with 28, 30, or 31-day months.

```dax
SalesTwoMonthsAgo = CALCULATE(
    SUM(Sales[Sales amount]),
    EDATE(Sales[Date], -2)
)
```

This formula provides the total sales from two months prior to the current context.


### Quick Time-Shifting Functions

Power BI also includes a number of functions to quickly achieve common comparisons. In some cases they duplicate the functionality of more general functions like `DATEADD`, but they can be used to reduce the complexity of your DAX expressions.

#### `SAMEPERIODLASTYEAR`, `SAMEPERIODLASTQUARTER`

These functions returns the same set of dates from the previous year or quarter.

For example, if your current context is March 1 to March 31 of 2023, `SAMEPERIODLASTYEAR` will provide you with dates from March 1 to March 31 of 2022. They are useful for "like-for-like" comparisons. Let's say you're analyzing sales for a specific week or month this year and want to compare it to the exact same week or month from the previous year. 

```dax
PreviousYearSales = CALCULATE(
    SUM(Sales[Sales amount]),
    SAMEPERIODLASTYEAR(Dates[Date])
)
```

#### `PREVIOUSQUARTER` and `PREVIOUSYEAR`

These functions return the full set of dates for the *entire* previous period, regardless of the current context. It's useful when you want to compare a specific period's data against the entire data of the previous year. For instance, if you want to compare Q1 sales of 2023 against the total sales of 2022, you'd use `PREVIOUSYEAR`.

```dax
Sales Previous Year = CALCULATE(
    SUM(Sales[Amount]),
    PREVIOUSYEAR(Dates[Date])
)
```




### `DATESYTD`, `DATESQTD`, `DATESMTD`

These functions extract dates from the beginning of the respective period up to a certain date. They're versatile and can be coupled with other DAX functions for a wide range of analyses.

Imagine you have a sales table that also includes returned items. You want to calculate the net sales for the year-to-date, where net sales is defined as total sales minus total returns.

```dax
Net_YTD_Sales = CALCULATE(
    SUM(Sales[Amount]) - SUM(Sales[Returns]),
    DATESYTD(DateTable[Date])
)
```

Here, we subtract the returned items' value within the YTD timeframe. 




## Key Takeaways

- Time intelligence functions in Power BI enable powerful temporal analyses, and the comparison of measures across time periods
- Properly configured date tables are vital for accurate time intelligence
- Continuous date tables ensure representation for all dates, ensuring time intelligence functions will work correctly
- Always mark your date table in Power BI for optimal performance and compatibility
- The `CALENDAR` function provides a straightforward way to create a date table
- The `DATESBETWEEN` function allows for custom date range creation
- You can expand your date table by calculating new columns with functions like `STARTOFYEAR` or `STARTOFQUARTER`
- Power BI lacks a direct `STARTOFWEEK` function, but it can be derived using the `WEEKDAY` function
- Functions like `TOTALYTD` and `TOTALQTD` compute cumulative totals for specific time frames
- Time-based filtering, such as with `DATEADD` or `EDATE`, enables shifted period analyses
- Quick time-shifting functions like `SAMEPERIODLASTYEAR` simplify common time comparisons
- Functions like `DATESYTD` or `DATESQTD` extract dates from the start of a period up to a specific date